# 千帆 与 Weights & Biases

Weights & Biases 提供了一个平台，可以对大模型的调用过程进行可视化的展现，跟踪每次请求对资源的消耗等等，尤其是在开发 Agent 等涉及到复杂的模型调用时，可以帮助开发者更好地观察模型效果并进一步去改进模型。

本文将介绍如何利用 LangChain，将千帆SDK的调用导入 Weights & Biases。

## 1. 初始化

为了能够使用 Weights & Biases，需要先安装 wandb 包。

In [ ]:
!pip install wandb

之后在代码中我们需要开启 LangChain 对 WanDB 的 Tracing 功能。

In [ ]:
import os
import wandb
import qianfan

# 初始化千帆 SDK
os.environ["QIANFAN_AK"] = "your_ak"
os.environ["QIANFAN_SK"] = "your_sk"
# 开启 Langchain 对 WanDB 的 Tracing 功能
os.environ["LANGCHAIN_WANDB_TRACING"] = "true"
# 设置 WanDB 的项目名称
os.environ["WANDB_PROJECT"] = "my-wandb-llm-project"

## 2. 使用

使用方法与正常使用 LangChain 一致，唯一的区别是在调用模型时，需要用 wandb 进行包装。

在第一次使用时，需要填入 wandb 的 api key，请遵循输出的提示进行注册、获取 key 等操作。

In [ ]:
from langchain.chat_models import QianfanChatEndpoint

llm = QianfanChatEndpoint(model="ERNIE-Bot")

with wandb.init(project='langchain-tracing') as run:
    resp = llm.invoke("hello")

运行完成后，会输出该次运行的链接，可以在 wandb 的控制台上查看到 trace 信息。

## 3. 跟踪 Agent

这里为了展示一个更为复杂的用例，我们先创建一个 Agent，采用的是在 [这一 cookbook](https://github.com/baidubce/bce-qianfan-sdk/blob/main/cookbook/qianfan_agent/qianfan_agent.ipynb) 中所构建的利用 function call 实现的获取天气信息的 Agent，在此不再重复介绍。

这边也可以替换能任何通过 LangChain 实现的组件。

In [ ]:
from langchain.chat_models import QianfanChatEndpoint
from langchain.agents import load_tools, initialize_agent, AgentType
from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain.agents import tool
from langchain.agents.output_parsers import OpenAIFunctionsAgentOutputParser
from langchain.agents.format_scratchpad import format_to_openai_functions

llm = QianfanChatEndpoint(model="ERNIE-Bot") 
prompt = ChatPromptTemplate.from_messages([
    ("system", "You are very powerful assistant, but bad at get today's temperature of location."),
    ("user", "{input}"),
    MessagesPlaceholder(variable_name="agent_scratchpad")
])

FUNCTION_SCHEMA_GET_WEATHER = {
    "name": "get_current_weather",
    "description": "获得指定地点的天气",
    "parameters": {
        "type": "object",
        "properties": {
        "location": {
            "type": "string",
            "description": "省，市名，例如：河北省，石家庄"
        },
        "unit": {
            "type": "string",
            "enum": ["摄氏度", "华氏度"]
        }
        },
        "required": ["location"]
    }
}

@tool
def get_current_weather(location: str) -> str:
    """Returns current temperature of location."""
    return "25"

tools = [get_current_weather]

llm_with_tools = llm.bind(
    functions=[
        FUNCTION_SCHEMA_GET_WEATHER,
    ]
)
p = {
    "input": lambda x: x["input"],
    "agent_scratchpad": lambda x: format_to_openai_functions(x['intermediate_steps']),
} 
agent = p | prompt | llm_with_tools | OpenAIFunctionsAgentOutputParser()

创建完 Agent 后，同样的与正常使用基本一致，只需要将需要跟踪的部分用 wandb 对象进行包裹即可。

In [ ]:
from langchain.schema.agent import AgentFinish
intermediate_steps = []

with wandb.init(project='langchain-tracing') as run:
    while True:
        output = agent.invoke({
            "input": "上海市今天天气如何？",
            "intermediate_steps": intermediate_steps
        })
        
        if isinstance(output, AgentFinish):
            final_result = output.return_values["output"]
            break
        else:
            print(output.tool, output.tool_input)
            tool = {
                "get_current_weather": get_current_weather,
            }[output.tool]
            observation = tool.run(output.tool_input)
            intermediate_steps.append((output, observation))
print(final_result)

从输出中就可以拿到这次跟踪的链接，可以从控制台看到 Agent 运行的每一个步骤和结果，以及各类统计信息。

至此，我们就完成了千帆与 wandb 的结合，可以在 wandb 的控制台上看到千帆模型的执行结果，希望能对您大模型的开发带来帮助！